In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#Export
import os,uuid,sys,base64
import pandas as pd
import numpy as np
import GEOparse
import zipfile
import re

from flask import Flask,render_template,flash,redirect,url_for,request,session,jsonify,send_from_directory
from flask_ckeditor import CKEditor,upload_success,upload_fail
from flask_dropzone import Dropzone
from flask_wtf.csrf import validate_csrf
from wtforms import ValidationError
from markupsafe import escape
from werkzeug.utils import secure_filename
from werkzeug.datastructures import FileStorage
from matplotlib.figure import Figure
from Bio import Entrez


In [ ]:
Entrez.email = "464967383@qq.com"

In [ ]:
handle = Entrez.einfo() # returns a handle object
handle

In [ ]:
record = Entrez.read(handle)
record

In [ ]:
handle.close()

In [ ]:
query = 'Mus Musculus [Organism] AND Expression profiling by high throughput sequencing [DataSet Type]'

In [ ]:
handle = Entrez.esearch(db='gds', term=query, retmax=5, usehistory=True)

In [ ]:
result = Entrez.read(handle)

In [ ]:
handle.close()

In [ ]:
print(result)

In [ ]:
uid_regex = re.compile('[1-9]+0+([1-9]+[0-9]*)')
uid_regex

In [ ]:
gse_list = ['GSE' + uid_regex.match(uid).group(1) for uid in result['IdList']]
gse_list

In [24]:
gse.

{'title': ['Dementia subtype prediction models constructed by penalized regression methods for multiclass classification using serum microRNA expression data'],
 'geo_accession': ['GSE167559'],
 'status': ['Public on Oct 22 2021'],
 'submission_date': ['Feb 25 2021'],
 'last_update_date': ['Nov 03 2021'],
 'pubmed_id': ['34686734'],
 'summary': ['There are many subtypes of dementia, and identification of diagnostic biomarkers that are minimally-invasive, low-cost, and efficient is desired. Circulating microRNAs (miRNAs) have recently gained attention as easily accessible and non-invasive biomarkers. We conducted a comprehensive miRNA expression analysis of serum samples from 1348 Japanese dementia patients, composed of four subtypes—Alzheimer’s disease (AD), vascular dementia, dementia with Lewy bodies (DLB), and normal pressure hydrocephalus—and 246 control subjects. We used this data to construct dementia subtype prediction models based on penalized regression models with the multicl

In [71]:
# ls = ['GSE65391','GSE121239','GSE61635']
# ls = ['GSE17755']#'GSE15573',
ls = ['GSE20681']#'GSE188309','GSE65682','GSE42830','GSE42826',
for i in ls:
    while True:
        try:
            gse = GEOparse.get_GEO(geo=i, destdir="./df-based-biomarker-miner/upload_file/geo/", silent=False,include_data = True,annotate_gpl=True)
            break
        except OSError:
            continue
    expr = gse.pivot_and_annotate(values='VALUE', gpl=gse.gpls[list(gse.gpls.keys())[0]].table,annotation_column='GENE_SYMBOL',gpl_on='ID',gsm_on='ID_REF').set_index('GENE_SYMBOL').T
    expr = expr.merge(right=gse.phenotype_data,left_index=True,right_index=True)
    # clinical = gse.phenotype_data
    # df = pd.concat([expr,clinical],axis = 1)
    expr.T.to_csv(path_or_buf=f"./df-based-biomarker-miner/upload_file/geo/{i}.csv")

28-Dec-2022 06:04:24 DEBUG utils - Directory ./df-based-biomarker-miner/upload_file/geo/ already exists. Skipping.
28-Dec-2022 06:04:24 INFO GEOparse - File already exist: using local version.
28-Dec-2022 06:04:24 INFO GEOparse - Parsing ./df-based-biomarker-miner/upload_file/geo/GSE20681_family.soft.gz: 
28-Dec-2022 06:04:24 DEBUG GEOparse - DATABASE: GeoMiame
28-Dec-2022 06:04:24 DEBUG GEOparse - SERIES: GSE20681
28-Dec-2022 06:04:24 DEBUG GEOparse - PLATFORM: GPL4133
28-Dec-2022 06:04:25 WARNING GEOTypes - Detected duplicated columns in d GPL4133. Correcting.

28-Dec-2022 06:04:25 DEBUG GEOparse - SAMPLE: GSM518885
28-Dec-2022 06:04:25 DEBUG GEOparse - SAMPLE: GSM518886
28-Dec-2022 06:04:25 DEBUG GEOparse - SAMPLE: GSM518887
28-Dec-2022 06:04:25 DEBUG GEOparse - SAMPLE: GSM518888
28-Dec-2022 06:04:25 DEBUG GEOparse - SAMPLE: GSM518889
28-Dec-2022 06:04:25 DEBUG GEOparse - SAMPLE: GSM518890
28-Dec-2022 06:04:25 DEBUG GEOparse - SAMPLE: GSM518891
28-Dec-2022 06:04:25 DEBUG GEOparse - 

In [97]:
expr

,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,contact_institute,contact_address,contact_city,contact_state,contact_zip/postal_code,contact_country,supplementary_file,series_id,data_row_count,characteristics_ch1.7.additional diagnosis:
name,,,,,,,,,,,,,,,,,,,,,
GSM991986,0.683853,0.932607,0.848980,0.131172,0.880916,0.752335,0.672554,0.719074,0.176820,0.003874,...,ASRC-CUNY,85 St Nicholas Terrace,New York,NY,10031,USA,NONE,GSE40360,481917,NaN
GSM991987,0.733556,0.955674,0.830002,0.152967,0.892171,0.735081,0.664106,0.685440,0.222098,0.006231,...,ASRC-CUNY,85 St Nicholas Terrace,New York,NY,10031,USA,NONE,GSE40360,481917,NaN
GSM991988,0.590328,0.963023,0.848742,0.148265,0.947618,0.616499,0.711467,0.560839,0.287189,0.007169,...,ASRC-CUNY,85 St Nicholas Terrace,New York,NY,10031,USA,NONE,GSE40360,481917,NaN
GSM991989,0.638373,0.918614,0.813404,0.235173,0.882049,0.696077,0.706137,0.647228,0.197806,0.007881,...,ASRC-CUNY,85 St Nicholas Terrace,New York,NY,10031,USA,NONE,GSE40360,481917,NaN
GSM991990,0.613812,0.951447,0.828776,0.162109,0.896294,0.615303,0.703188,0.641617,0.225014,0.004760,...,ASRC-CUNY,85 St Nicholas Terrace,New York,NY,10031,USA,NONE,GSE40360,481917,NaN
GSM991991,0.720735,0.956555,0.876941,0.171313,0.903914,0.560531,0.634956,0.640860,0.213681,0.002364,...,ASRC-CUNY,85 St Nicholas Terrace,New York,NY,10031,USA,NONE,GSE40360,481917,NaN
GSM991992,0.524450,0.948158,0.833365,0.177087,0.841082,0.426121,0.704850,0.564299,0.130736,0.002744,...,ASRC-CUNY,85 St Nicholas Terrace,New York,NY,10031,USA,NONE,GSE40360,481917,NaN
GSM991993,0.624562,0.956308,0.820268,0.184909,0.841743,0.564454,0.634867,0.629126,0.206014,0.004523,...,ASRC-CUNY,85 St Nicholas Terrace,New York,NY,10031,USA,NONE,GSE40360,481917,NaN
GSM991994,0.674623,0.968683,0.870027,0.141749,0.866191,0.573977,0.689411,0.587390,0.213257,0.000000,...,ASRC-CUNY,85 St Nicholas Terrace,New York,NY,10031,USA,NONE,GSE40360,481917,NaN


In [69]:
gse.gpls[list(gse.gpls.keys())[0]].table

,ID,COL,ROW,NAME,SPOT_ID,CONTROL_TYPE,REFSEQ,GB_ACC,GENE,GENE_SYMBOL,...,ENSEMBL_ID,TIGR_ID,ACCESSION_STRING,CHROMOSOMAL_LOCATION,CYTOBAND,DESCRIPTION,GO_ID,SEQUENCE,SPOT_ID.1,ORDER
0,1,266,170,GE_BrightCorner,GE_BrightCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2,266,168,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,3,266,166,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,4,266,164,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,5,266,162,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45215,45216,1,9,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45216
45216,45217,1,7,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45217
45217,45218,1,5,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45218
45218,45219,1,3,GE_BrightCorner,GE_BrightCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45219


In [5]:
print()
print("GSM example:")
for gsm_name, gsm in gse.gsms.items():
    print("Name: ", gsm_name)
    print("Metadata:",)
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print ("Table data:",)
    print (gsm.table.head())
    break

print()
print("GPL example:")
for gpl_name, gpl in gse.gpls.items():
    print("Name: ", gpl_name)
    print("Metadata:",)
    for key, value in gpl.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print("Table data:",)
    print(gpl.table.head())
    break


GSM example:
Name:  GSM3332260
Metadata:
 - title : M1Fovea1
 - geo_accession : GSM3332260
 - status : Public on Feb 06 2019
 - submission_date : Aug 14 2018
 - last_update_date : Feb 06 2019
 - type : SRA
 - channel_count : 1
 - source_name_ch1 : Macaque 1 retina
 - organism_ch1 : Macaca fascicularis
 - taxid_ch1 : 9541
 - characteristics_ch1 : tissue: Fovea retina
 - treatment_protocol_ch1 : Eyes from male and female macaques (Macaca fascicularis) of 3-9 years of age were generously provided by Biomere (Worcester, MA), Mikhail Papisov,Curtis Cetrulo, or Tatsuo Kawai (Massachusetts General Hospital) or purchased from Toxikon Corporation (Bedford, MA). For sequencing and labeling, macaque eyes were collected either under deep anesthesia or 15-45 min postmortem, then immediately stored as either whole globes in ice-cold Ames solution (Sigma-Aldrich; equilibrated with 95% O2/5% CO2 for all use) or as posterior poles in room temperature Ames solution following a rapid hemisection to remo

In [70]:
gse.gpls[list(gse.gpls.keys())[0]].columns

,description
ID,Agilent feature number
COL,Column
ROW,Row
NAME,NAME
SPOT_ID,Spot identifier
CONTROL_TYPE,Control type
REFSEQ,RefSeqAccession
GB_ACC,"GenBankAccession LINK_PRE:""http://www.ncbi.nlm..."
GENE,"Entrez Gene ID LINK_PRE:""http://www.ncbi.nlm.n..."
GENE_SYMBOL,Gene Symbol


In [64]:
gse.gpls[list(gse.gpls.keys())[0]].table

,ID,COL,ROW,NAME,SPOT_ID,CONTROL_TYPE,REFSEQ,GB_ACC,GENE,GENE_SYMBOL,...,ENSEMBL_ID,TIGR_ID,ACCESSION_STRING,CHROMOSOMAL_LOCATION,CYTOBAND,DESCRIPTION,GO_ID,SEQUENCE,SPOT_ID.1,ORDER
0,1,266,170,GE_BrightCorner,GE_BrightCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2,266,168,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,3,266,166,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,4,266,164,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,5,266,162,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45215,45216,1,9,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45216
45216,45217,1,7,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45217
45217,45218,1,5,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45218
45218,45219,1,3,GE_BrightCorner,GE_BrightCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45219


In [16]:
gse.pivot_and_annotate(values='VALUE', gpl=gse.gpls[list(gse.gpls.keys())[0]].table,annotation_column='Gene Symbol',gpl_on='ID',gsm_on='ID_REF').set_index('Gene Symbol').T

KeyError: "['Gene Symbol'] not in index"

In [47]:
gse.phenotype_data

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,taxid_ch1,...,contact_department,contact_institute,contact_address,contact_city,contact_state,contact_zip/postal_code,contact_country,supplementary_file,series_id,data_row_count
GSM5677392,AA242,GSM5677392,Public on Oct 17 2022,Nov 05 2021,Oct 17 2022,RNA,1,Total RNA from peripheral blood,Homo sapiens,9606,...,"IMIM,",MARGenomics,Dr Aiguader 88,Barcelona,Catalunya,08003,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5677...,GSE188309,24351
GSM5677393,AA325,GSM5677393,Public on Oct 17 2022,Nov 05 2021,Oct 17 2022,RNA,1,Total RNA from peripheral blood,Homo sapiens,9606,...,"IMIM,",MARGenomics,Dr Aiguader 88,Barcelona,Catalunya,08003,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5677...,GSE188309,24351
GSM5677394,AC209,GSM5677394,Public on Oct 17 2022,Nov 05 2021,Oct 17 2022,RNA,1,Total RNA from peripheral blood,Homo sapiens,9606,...,"IMIM,",MARGenomics,Dr Aiguader 88,Barcelona,Catalunya,08003,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5677...,GSE188309,24351
GSM5677395,AC270,GSM5677395,Public on Oct 17 2022,Nov 05 2021,Oct 17 2022,RNA,1,Total RNA from peripheral blood,Homo sapiens,9606,...,"IMIM,",MARGenomics,Dr Aiguader 88,Barcelona,Catalunya,08003,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5677...,GSE188309,24351
GSM5677396,AF264,GSM5677396,Public on Oct 17 2022,Nov 05 2021,Oct 17 2022,RNA,1,Total RNA from peripheral blood,Homo sapiens,9606,...,"IMIM,",MARGenomics,Dr Aiguader 88,Barcelona,Catalunya,08003,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5677...,GSE188309,24351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM5677585,SR312,GSM5677585,Public on Oct 17 2022,Nov 05 2021,Oct 17 2022,RNA,1,Total RNA from peripheral blood,Homo sapiens,9606,...,"IMIM,",MARGenomics,Dr Aiguader 88,Barcelona,Catalunya,08003,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5677...,GSE188309,24351
GSM5677586,ST195,GSM5677586,Public on Oct 17 2022,Nov 05 2021,Oct 17 2022,RNA,1,Total RNA from peripheral blood,Homo sapiens,9606,...,"IMIM,",MARGenomics,Dr Aiguader 88,Barcelona,Catalunya,08003,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5677...,GSE188309,24351
GSM5677587,TG198,GSM5677587,Public on Oct 17 2022,Nov 05 2021,Oct 17 2022,RNA,1,Total RNA from peripheral blood,Homo sapiens,9606,...,"IMIM,",MARGenomics,Dr Aiguader 88,Barcelona,Catalunya,08003,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5677...,GSE188309,24351
GSM5677588,VC189,GSM5677588,Public on Oct 17 2022,Nov 05 2021,Oct 17 2022,RNA,1,Total RNA from peripheral blood,Homo sapiens,9606,...,"IMIM,",MARGenomics,Dr Aiguader 88,Barcelona,Catalunya,08003,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5677...,GSE188309,24351


In [18]:
gse.gsms[list(gse.gsms.keys())[0]].columns

,description
ID_REF,
VALUE,Global normalized signal intensity


In [ ]:
gses=gse.download_supplementary_files()

In [9]:
gse.gsms[list(gse.gsms.keys())[0]].metadata

{'title': ['M1Fovea1'],
 'geo_accession': ['GSM3332260'],
 'status': ['Public on Feb 06 2019'],
 'submission_date': ['Aug 14 2018'],
 'last_update_date': ['Feb 06 2019'],
 'type': ['SRA'],
 'channel_count': ['1'],
 'source_name_ch1': ['Macaque 1 retina'],
 'organism_ch1': ['Macaca fascicularis'],
 'taxid_ch1': ['9541'],
 'characteristics_ch1': ['tissue: Fovea retina'],
 'treatment_protocol_ch1': ['Eyes from male and female macaques (Macaca fascicularis) of 3-9 years of age were generously provided by Biomere (Worcester, MA), Mikhail Papisov,Curtis Cetrulo, or Tatsuo Kawai (Massachusetts General Hospital) or purchased from Toxikon Corporation (Bedford, MA). For sequencing and labeling, macaque eyes were collected either under deep anesthesia or 15-45 min postmortem, then immediately stored as either whole globes in ice-cold Ames solution (Sigma-Aldrich; equilibrated with 95% O2/5% CO2 for all use) or as posterior poles in room temperature Ames solution following a rapid hemisection to r

In [ ]:
gse = GEOparse.get_GEO(filepath="./upload_file/geo/GSE142000_family.soft.gz")

In [48]:
gse.get_type()

'Expression profiling by array'

In [ ]:
# annotate with GPL
lfc_result_annotated = pivoted_samples.reset_index().merge(gse.gpls['GPL570'].table[["ID", "Gene Symbol"]],
                                left_on='index', right_on="ID").set_index('index')
del lfc_result_annotated["ID"]
# remove probes without ENTREZ
lfc_result_annotated = lfc_result_annotated.dropna(subset=["Gene Symbol"])
# remove probes with more than one gene assigned
lfc_result_annotated = lfc_result_annotated[~lfc_result_annotated.ENTREZ_GENE_ID.str.contains("///")]
# for each gene average LFC over probes
lfc_result_annotated = lfc_result_annotated.groupby("Gene Symbol").median()